In [1]:
# VLM related
from transformers import Qwen2_5_VLForConditionalGeneration, AutoProcessor
from qwen_vl_utils import process_vision_info
import torch
from PIL import Image
# OmniGibson related
from envs.base_env import BaseEnvironment
from omnigibson.utils.ui_utils import KeyboardRobotController
from utils.debug import setup_debug_keys

/home/litiangong/miniconda3/envs/omnigibson/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# 初始化Qwen VL模型
model = Qwen2_5_VLForConditionalGeneration.from_pretrained(
    "Qwen/Qwen2.5-VL-3B-Instruct", torch_dtype="auto", device_map="auto"
)
model.eval()

processor = AutoProcessor.from_pretrained("Qwen/Qwen2.5-VL-3B-Instruct")

Loading checkpoint shards: 100%|██████████| 2/2 [00:00<00:00,  2.44it/s]
Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.


In [3]:
def inference_with_vlm(rgb_tensor, proprio_tensor, model, processor, process_vision_info, prompt_template=None):
    """
    使用视觉语言模型对机器人观测进行推理
    
    参数:
        rgb_tensor (torch.Tensor): RGB图像张量，形状为(H, W, C)
        proprio_tensor (torch.Tensor): 机器人本体感知数据张量
        model: VLM模型
        processor: VLM处理器
        process_vision_info: 处理视觉信息的函数
        prompt_template (str, optional): 可选的提示模板，如果为None则使用默认模板
        
    返回:
        str: 模型生成的文本结果
    """
    # 处理RGB图像
    if rgb_tensor.shape[2] == 4:
        rgb_tensor = rgb_tensor[:, :, :3]
    # 转换为PIL图像
    rgb_img = Image.fromarray(rgb_tensor.numpy())
    # 本体感知张量转换为numpy
    proprio_vector = proprio_tensor.numpy()
    
    # 构建提示文本
    prompt = f"Based on the robot's proprioceptive data:{proprio_vector}, determine the next action."
    if prompt_template:
        prompt = prompt_template.format(proprio=proprio_vector)
    
    # 定义输入消息
    messages = [
        {
            "role": "user",
            "content": [
                {"type": "image", "image": rgb_img},
                {"type": "text", "text": prompt}
            ],
        }
    ]
    
    # 应用聊天模板
    text = processor.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)

    # 处理视觉信息
    image_inputs, video_inputs = process_vision_info(messages)

    # 准备模型输入
    inputs = processor(
        text=[text],
        images=image_inputs,
        videos=video_inputs,
        padding=True,
        return_tensors="pt"
    )

    # 将输入移至适当的设备
    inputs = {k: v.to(model.device) for k, v in inputs.items()}

    # 执行推理
    with torch.no_grad():
        generated_ids = model.generate(**inputs, max_new_tokens=128)

    # 解码生成的输出
    generated_ids_trimmed = [
        out_ids[len(in_ids):] for in_ids, out_ids in zip(inputs["input_ids"], generated_ids)
    ]
    output_text = processor.batch_decode(
        generated_ids_trimmed, skip_special_tokens=True, clean_up_tokenization_spaces=False
    )

    # 返回生成的文本
    return output_text[0]

In [4]:
# initialize og env
env = BaseEnvironment(configs="config/scene_config.yaml")
robot = env.robots[0]
# initialize robot controller
keyboard_controller = KeyboardRobotController(robot)
setup_debug_keys(keyboard_controller, robot, env)

count = 0
while True:
    count += 1
    action = keyboard_controller.get_teleop_action()
    obs, reward, terminated, truncated, info = env.step(action)
    # fetch rgb and proprio
    rgb_tensor = obs[robot.name][f"{robot.name}:eyes:Camera:0"]["rgb"]
    proprio_tensor = obs[robot.name]["proprio"]
    # shape of rgb_tensor
    print(rgb_tensor.shape)
    # shape of proprio_tensor
    print(proprio_tensor.shape)
    # Remove the alpha channel if present
    if rgb_tensor.shape[2] == 4:
        rgb_tensor = rgb_tensor[:, :, :3]
    # Convert to PIL Image
    rgb_img = Image.fromarray(rgb_tensor.numpy())
    # Priprio tensor to numpy
    proprio_vector = proprio_tensor.numpy()
    # Define the input message
    result = inference_with_vlm(
        rgb_tensor=rgb_tensor,
        proprio_tensor=proprio_tensor,
        model=model,
        processor=processor,
        process_vision_info=process_vision_info,
        prompt_template="我看到了机器人环境的图像，机器人的状态数据为:{proprio}，请分析这个场景并建议下一步操作。"
    )
    print(result)


[INFO] [omnigibson.simulator] ----- Starting OmniGibson. This will take 10-30 seconds... -----


正在创建环境，这可能需要一些时间...
[Warning] [omni.isaac.kit] Interactive python shell detected but ISAAC_JUPYTER_KERNEL was not set. Problems with asyncio may occur
[Warning] [omni.isaac.kit] Please use Isaac Sim Python 3 kernel instead of the default Python 3 Kernel
Starting kit application with the following args:  ['/home/litiangong/miniconda3/envs/omnigibson/lib/python3.10/site-packages/isaacsim/exts/omni.isaac.kit/omni/isaac/kit/simulation_app.py', '/home/litiangong/miniconda3/envs/omnigibson/lib/python3.10/site-packages/isaacsim/apps/omnigibson_4_1_0.kit', '--/app/tokens/exe-path=/home/litiangong/miniconda3/envs/omnigibson/lib/python3.10/site-packages/omni', '--/persistent/app/viewport/displayOptions=3094', '--/rtx/materialDb/syncLoads=True', '--/rtx/hydra/materialSyncLoads=True', '--/omni.kit.plugin/syncUsdLoads=True', '--/app/renderer/resolution/width=1280', '--/app/renderer/resolution/height=720', '--/app/window/width=1440', '--/app/window/height=900', '--/renderer/multiGpu/enabled=False', 

[INFO] [omni.kit.telemetry.impl.sentry_extension] sentry is disabled for external build
[INFO] [omni.kit.telemetry.impl.sentry_extension] sentry is disabled for external build


[0.858s] [ext: omni.kit.telemetry-0.5.0] startup
[0.881s] [ext: omni.kit.loop-isaac-1.2.0] startup
[0.883s] [ext: omni.kit.test-0.0.0] startup
[0.905s] [ext: omni.appwindow-1.1.8] startup
2025-04-27 10:09:55 s] [Warning] [carb.windowing-glfw.plugin] GLFW initialization failed.
2025-04-27 10:09:55 s] [Warning] [carb] Failed to startup plugin carb.windowing-glfw.plugin (interfaces: [carb::windowing::IGLContext v1.0],[carb::windowing::IWindowing v1.4]) (impl: carb.windowing-glfw.plugin)
[0.908s] [ext: omni.kit.renderer.core-1.0.1] startup
2025-04-27 10:09:55 s] [Warning] [carb.windowing-glfw.plugin] GLFW initialization failed.
2025-04-27 10:09:55 s] [Warning] [carb] Failed to startup plugin carb.windowing-glfw.plugin (interfaces: [carb::windowing::IGLContext v1.0],[carb::windowing::IWindowing v1.4]) (impl: carb.windowing-glfw.plugin)
2025-04-27 10:09:55 s] [Warning] [carb.windowing-glfw.plugin] GLFW initialization failed.
2025-04-27 10:09:55 s] [Warning] [carb] Failed to startup plugin ca

[DEBUG] [AutoNode] Defining data type 'any' as 'Any'
[DEBUG] [AutoNode] Defining data type 'bool' as 'Bool' and array 'BoolArray
[DEBUG] [AutoNode] Defining data type 'bundle' as 'Bundle'
[DEBUG] [AutoNode] Defining data type 'colord[3]' as 'Color3d' and array 'Color3dArray
[DEBUG] [AutoNode] Defining data type 'colorf[3]' as 'Color3f' and array 'Color3fArray
[DEBUG] [AutoNode] Defining data type 'colorh[3]' as 'Color3h' and array 'Color3hArray
[DEBUG] [AutoNode] Defining data type 'colord[4]' as 'Color4d' and array 'Color4dArray
[DEBUG] [AutoNode] Defining data type 'colorf[4]' as 'Color4f' and array 'Color4fArray
[DEBUG] [AutoNode] Defining data type 'colorh[4]' as 'Color4h' and array 'Color4hArray
[DEBUG] [AutoNode] Defining data type 'double' as 'Double' and array 'DoubleArray
[DEBUG] [AutoNode] Defining data type 'double[2]' as 'Double2' and array 'Double2Array
[DEBUG] [AutoNode] Defining data type 'double[3]' as 'Double3' and array 'Double3Array
[DEBUG] [AutoNode] Defining data t

...

[INFO] [omnigibson.simulator] ---------- Welcome to OmniGibson! ----------


2025-04-27 10:10:06 [11,061ms] [Warning] [omni.syntheticdata.plugin] OgnSdPostRenderVarToHost : rendervar copy from texture directly to host buffer is counter-performant. Please use copy from texture to device buffer first.
2025-04-27 10:10:06 [11,092ms] [Warning] [omni.fabric.plugin] removePath called on non-existent path /Render/RenderProduct_Replicator/PostRender/SDGPipeline/RenderProduct_Replicator_GpuInteropEntry 


                   ____________
                  /          / \
                 /          / /__
                /          / /  /\
               /__________/ /__/  \
               \   _____  \ \__\  /
                \  \  / \  \ \_/ /
                 \  \/___\  \   /
                  \__________\_/  
       ___                  _  ____ _ _                     
      / _ \ _ __ ___  _ __ (_)/ ___(_) |__  ___  ___  _ __  
     | | | | '_ ` _ \| '_ \| | |  _| | '_ \/ __|/ _ \| '_ \ 
     | |_| | | | | | | | | | | |_| | | |_) \__ \ (_) | | | |
      \___/|_| |_| |_

[INFO] [omnigibson.simulator] Imported scene 0.


2025-04-27 10:10:08 [13,099ms] [Warning] [carb] Failed to startup plugin carb.windowing-glfw.plugin (interfaces: [carb::windowing::IGLContext v1.0],[carb::windowing::IWindowing v1.4]) (impl: carb.windowing-glfw.plugin)
期望的物体总数: 7
分配给各类别的数量: [1, 1, 1, 1, 1, 1, 1]
桌子包围盒中心: tensor([ 1.4889, -0.5252,  0.3084])
桌子包围盒尺寸: tensor([1.1380, 0.7617, 0.6504])
桌子高度: 0.6335694789886475
桌子朝向: tensor([0.0000, 0.0000, 0.7071, 0.7071])
total_grid_cells: 28
桌面尺寸: 1.0779612064361572 x 0.7016814351081848
网格大小: 0.15 x 0.15
网格数量: 7 x 4 = 28
占用率: 0.8
可用位置数量: 22
生成了 22 个位置点
根据网格和占用率生成的可用位置数量: 22
可用位置(22)多于期望物体总数(7)，可以补充更多物体
最终的物体分配数量: [4, 3, 3, 3, 3, 3, 3]
最终的物体总数: 22
已生成 22 个物品配置，使用位置数量: 22
2025-04-27 10:10:10 [15,898ms] [Warning] [carb] Failed to startup plugin carb.windowing-glfw.plugin (interfaces: [carb::windowing::IGLContext v1.0],[carb::windowing::IWindowing v1.4]) (impl: carb.windowing-glfw.plugin)
成功添加了 22 个动态物体
等待场景稳定...
场景稳定完成
已设置机器人初始关节位置
已将当前关节位置设为默认重置位置
环境创建完成！
torch.Size([480, 640, 4])
torch.Siz

: 